### Worst use of Window Functions

In [1]:
# Create Spark Session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Worst use of Window Function") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/31 17:18:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark.sparkContext.defaultParallelism

8

In [2]:
# Set Spark default config
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [3]:
# Read example data set
from pyspark.sql import functions as F
import pandas as pd

df = spark.read.parquet("hdfs://namenode:9000/input/data/sales.parquet")

In [4]:
df.explain()

== Physical Plan ==
*(1) Scan ExistingRDD[transacted_at#0,trx_id#1L,retailer_id#2L,description#3,amount#4,city_id#5]




In [5]:
df.withColumn("partition_id", F.spark_partition_id()).groupBy("partition_id").count().show()

+------------+-------+
|partition_id|  count|
+------------+-------+
|           6|  53999|
|           3|1048576|
+------------+-------+



In [6]:
df.show()

24/10/26 13:16:07 WARN TaskSetManager: Stage 3 contains a task of very large size (8828 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+----------+-----------+--------------------+-------+------------+
|       transacted_at|    trx_id|retailer_id|         description| amount|     city_id|
+--------------------+----------+-----------+--------------------+-------+------------+
|2017-11-24T19:00:...|1995601912| 2077350195|Walgreen       11-25| 197.23|2.16510448E8|
|2017-11-24T19:00:...|1734117021|  644879053|unkn    ppd id: 7...|   8.58|  9.302599E8|
|2017-11-24T19:00:...|1734117022|  847200066|Wal-Mart  ppd id:...|1737.26|1.64641549E9|
|2017-11-24T19:00:...|1734117030| 1953761884|Home Depot     pp...|  384.5|2.87177632E8|
|2017-11-24T19:00:...|1734117089| 1898522855| Target        11-25|  66.33| 1.8555305E9|
|2017-11-24T19:00:...|1734117117|  997626433|Sears  ppd id: 85...| 298.87| 9.5734701E8|
|2017-11-24T19:00:...|1734117123| 1953761884|unkn   ppd id: 15...|  19.55| 4.5522088E7|
|2017-11-24T19:00:...|1734117152| 1429095612|Ikea     arc id: ...|   9.39|1.26854131E9|
|2017-11-24T19:00:...|1734117153

In [7]:
df.write.format("noop").mode("overwrite").save()

In [8]:
# Use row_number() to generate ids
df_with_row_num = df.withColumn("_id", F.expr("row_number() over (order by null)"))

# Write the dataset in noop for performance benchmarking
df_with_row_num.write.format("noop").mode("overwrite").save()

24/10/31 17:23:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/31 17:23:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/31 17:23:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/31 17:23:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/31 17:23:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [ ]:
df_with_row_num.show()

In [9]:
# Validate the number of partitions
df_with_row_num.rdd.getNumPartitions()

24/10/31 17:24:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/31 17:24:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/31 17:24:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/31 17:24:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/31 17:24:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


1

In [10]:
# Use Spark in-built monotonically_increasing_id to generate ids
df_with_incr_id = df.withColumn("_id", F.monotonically_increasing_id())

# Write the dataset in noop for performance benchmarking
df_with_incr_id.write.format("noop").mode("overwrite").save()

In [ ]:
df_with_incr_id.show()

In [11]:
# Validate the number of partitions
df_with_incr_id.rdd.getNumPartitions()

8

In [12]:
spark.stop()